In [ ]:
# Imports!

import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import re
import nltk
import joblib

from pandas import DataFrame as df
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB as NaiveBayes
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D, Flatten, Conv1D
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.models import load_model
nltk.download('wordnet', quiet = True)
nltk.download('averaged_perceptron_tagger', quiet = True)

# TASK 1

## (1) Load dataset 

In [ ]:
# Load dataset.
complete_data = pd.read_csv("22203569.csv", index_col = 0)
# Display the data.
complete_data

In [ ]:
# Two categories
complete_data['category'].nunique()
complete_data.category.unique()

## (2) Most common terms

In [ ]:
data = complete_data[['category', 'short_description']]
data

In [ ]:
# It is an imbalanced problem
data['category'].value_counts()

In [ ]:
# Finding words more common for each target variable.
parenting_class = data[data['category'] == 'PARENTING']['short_description']
impact_class = data[data['category'] == 'IMPACT']['short_description']

In [ ]:
# Parenting class analysis of most common words.
parenting_class.isnull().sum()
parenting_class = parenting_class.fillna('')
parenting_class.isnull().sum()

In [ ]:
parenting_class

In [ ]:
df_parenting_class = pd.DataFrame(parenting_class, columns = ["short_description"])
df_parenting_class

In [ ]:
# Expand contractions
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

# Apply expansion to the 'short_description' column
df_parenting_class["short_description"] = df_parenting_class["short_description"].apply(expand_contractions)
df_parenting_class

In [ ]:
# Analyze the parenting class
parenting_class = df_parenting_class["short_description"]
parenting_class

In [ ]:
vectorizer = CountVectorizer(stop_words = 'english')
words_in_parenting_class = vectorizer.fit_transform(parenting_class)

In [ ]:
# Calculate the counts of words in parenting class.
tokens_and_counts_parenting_class = zip(vectorizer.get_feature_names_out(), np.asarray(words_in_parenting_class.sum(axis = 0)).ravel())

In [ ]:
df_tokens_parenting_class = pd.DataFrame(tokens_and_counts_parenting_class, columns = ['Token', 'Count'])
df_tokens_parenting_class

In [ ]:
# Check the words counts
df_tokens_parenting_class.sort_values('Count', ascending = False, inplace = True)
df_tokens_parenting_class.reset_index(inplace = True, drop = True)
df_tokens_parenting_class

In [ ]:
# Find the most common words in parenting class.
most_common_tokens_parenting_class = df_tokens_parenting_class.nlargest(columns = 'Count', n = 15)
most_common_tokens_parenting_class

In [ ]:
# Find the least common words in parenting class.
least_common_tokens_parenting_class = df_tokens_parenting_class.nsmallest(columns = 'Count', n = 15)
least_common_tokens_parenting_class

In [ ]:
# Display the diagram of the most common words and least common words in parenting class.
fig, axes = plt.subplots(2, 1, figsize = (20, 8))
sns.barplot(ax = axes[0], data = most_common_tokens_parenting_class, x = 'Token', y = 'Count')
sns.barplot(ax = axes[1], data = least_common_tokens_parenting_class, x = 'Token', y = 'Count')
axes[0].set(ylabel = 'Counts', xlabel = 'Words', title = '%d Most Common Words After Stop Word Removal' % 20)
axes[1].set(ylabel = 'Counts', xlabel = 'Words', title = '%d Least Common Words After Stop Word Removal' % 20)
plt.tight_layout()

In [ ]:
# Impact class analysis of most common words.
impact_class.isnull().sum()
impact_class = impact_class.fillna('')
impact_class.isnull().sum()

In [ ]:
# Display the content of impact class
impact_class

In [ ]:
df_impact_class = pd.DataFrame(impact_class, columns = ["short_description"])
df_impact_class

In [ ]:
# Expand contractions
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

# Apply expansion to the 'short_description' column
df_impact_class["short_description"] = df_impact_class["short_description"].apply(expand_contractions)
df_impact_class

In [ ]:
# Analyze the impact class
impact_class = df_impact_class["short_description"]
impact_class

In [ ]:
vectorizer = CountVectorizer(stop_words = 'english')
words_in_impact_class = vectorizer.fit_transform(impact_class)

In [ ]:
# Calculate the counts of words in impact class.
tokens_and_counts_impact_class = zip(vectorizer.get_feature_names_out(), np.asarray(words_in_impact_class.sum(axis = 0)).ravel())

In [ ]:
df_tokens_impact_class = pd.DataFrame(tokens_and_counts_impact_class, columns = ['Token', 'Count'])
df_tokens_impact_class

In [ ]:
# Check the words counts
df_tokens_impact_class.sort_values('Count', ascending = False, inplace = True)
df_tokens_impact_class.reset_index(inplace = True, drop = True)
df_tokens_impact_class

In [ ]:
# Find the most common words in impact class.
most_common_tokens_impact_class = df_tokens_impact_class.nlargest(columns = 'Count', n = 15)
most_common_tokens_impact_class

In [ ]:
# Find the least common words in impact class.
least_common_tokens_impact_class = df_tokens_impact_class.nsmallest(columns = 'Count', n = 15)
least_common_tokens_impact_class

In [ ]:
# Display the diagram of the most common words and least common words in impact class.
fig, axes = plt.subplots(2, 1, figsize = (20, 8))
sns.barplot(ax = axes[0], data = most_common_tokens_impact_class, x = 'Token', y = 'Count')
sns.barplot(ax = axes[1], data = least_common_tokens_impact_class, x = 'Token', y = 'Count')
axes[0].set(ylabel = 'Counts', xlabel = 'Words', title = '%d Most Common Words After Stop Word Removal' % 20)
axes[1].set(ylabel = 'Counts', xlabel = 'Words', title = '%d Least Common Words After Stop Word Removal' % 20)
plt.tight_layout()

## (3) Analyse Other Features

In [ ]:
# Firstly, the category - parenting
parenting_class

In [ ]:
# Build dataframe
df_parenting_class = pd.DataFrame(parenting_class, columns = ["category", "short_description", "sentence_length"])
df_parenting_class["category"] = "PARENTING"
df_parenting_class['sentence_length'] = df_parenting_class["short_description"].apply(lambda x: len(x.split()))
df_parenting_class

In [ ]:
# describe the data analysis
category_status_parenting = df_parenting_class.groupby('category')['sentence_length'].describe()
category_status_parenting

In [ ]:
# Secondly, the category - impact
impact_class

In [ ]:
# Build dataframe
df_impact_class = pd.DataFrame(impact_class, columns = ["category", "short_description", "sentence_length"])
df_impact_class["category"] = "IMPACT"
df_impact_class['sentence_length'] = df_impact_class["short_description"].apply(lambda x: len(x.split()))
df_impact_class

In [ ]:
# describe the data analysis
category_status_impact = df_impact_class.groupby('category')['sentence_length'].describe()
category_status_impact

## (4) Check the blank value and drop

In [ ]:
complete_data

In [ ]:
# subtract the needed data.
complete_data = complete_data.loc[:, ['category', 'short_description']]
complete_data

In [ ]:
# Drop the blank values.
complete_data = complete_data.dropna()
complete_data

In [ ]:
# Check if there are blank values in the dataset.
complete_data['category'].isnull().sum()
complete_data['short_description'].isnull().sum()

In [ ]:
# Comment:
'''
There are 275 blank values in the column of category and short_description, so I
dropped them.And because there is no int type values in the dataset, so I did not
check the outliers.
'''

# TASK 2

## (1) Split the dataset

In [ ]:
# Dataset
complete_data

In [ ]:
X = complete_data['short_description']
y = complete_data['category']

In [ ]:
# Split the dataset
X_train_plus_valid, X_test, y_train_plus_valid, y_test = train_test_split(X, y, random_state=0, test_size = 0.30, train_size = 0.7)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_plus_valid, y_train_plus_valid, random_state=0, test_size = 0.199/0.7, train_size = 0.5/0.7)

In [ ]:
X_train

In [ ]:
y_train_data = pd.DataFrame(y_train, columns = ['category'])
y_train_data
y_valid_data = pd.DataFrame(y_valid, columns = ['category'])
y_valid_data

In [ ]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

In [ ]:
# Export the three csv.
X_train.to_csv('train.csv', index=True)
X_valid.to_csv('valid.csv', index=True)
X_test.to_csv('test.csv', index=True)

In [ ]:
# Export y csv.
y_train.to_csv('y_train.csv', index=True)
y_valid.to_csv('y_valid.csv', index=True)
y_test.to_csv('y_test.csv', index=True)

## (2) Load train.csv and valid.csv

In [ ]:
# Load the train.csv and valid.csv.
train_data = pd.read_csv('train.csv', index_col=0)
valid_data = pd.read_csv('valid.csv', index_col=0)
train_data

In [ ]:
# Check the blank value in train.csv.
train_data['short_description'].isnull().sum()
#train_data['short_description'] = train_data['short_description'].fillna('')
#train_data['short_description'].isnull().sum()

In [ ]:
# Check the blank value in valid.csv.
valid_data['short_description'].isnull().sum()
#valid_data['short_description'] = valid_data['short_description'].fillna('')
#valid_data['short_description'].isnull().sum()

In [ ]:
# Expand contractions
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

# Apply expansion to the 'short_description' column
train_data["short_description"] = train_data["short_description"].apply(expand_contractions)
train_data
valid_data["short_description"] = valid_data["short_description"].apply(expand_contractions)
valid_data

In [ ]:
# Prepare the data for classification.
X_train = train_data['short_description']
y_train = y_train_data['category']
X_valid = valid_data['short_description']
y_valid = y_valid_data['category']
X_train

In [ ]:
# Generating a bag-of-words model

In [ ]:
# Preprocess the text data.
vectorizer = CountVectorizer(stop_words = 'english')
vectorizer.fit(X_train)

In [ ]:
X_train_matrix = vectorizer.transform(X_train)
X_valid_matrix = vectorizer.transform(X_valid)

In [ ]:
X_train_matrix
X_valid_matrix

In [ ]:
X_train_matrix.shape[1]

In [ ]:
print(X_train_matrix.shape)
print(X_valid_matrix.shape)

## (3) Build models using two classifiers

In [ ]:
y_train

In [ ]:
# Check the blank value in y_train.
y_train.isnull().sum()
#y_train = y_train.fillna('')
#y_train.isnull().sum()

In [ ]:
# Use the DecisionTree to build a model.
# Decision Tree is a versatile classifier that can handle both numerical and 
# categorical data. It builds a tree-like model of decisions and their possible 
# consequences. Decision Trees are interpretable and can capture complex interactions 
# between features. They can be particularly effective in text classification tasks
# when combined with appropriate feature extraction techniques such as TF-IDF or 
# word embeddings.
tree = DecisionTreeClassifier()
tree.fit(X_train_matrix, y_train)

In [ ]:
# Change X and y to the dataset I want to evaluate the results in:
X_cm = X_train_matrix
y_true_labels = y_train
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Save the model
joblib.dump(tree, 'DecisionTree_Model.joblib')

In [ ]:
y_pred

In [ ]:
y_train.value_counts()

In [ ]:
# y_pred is a numpy array. It needs to be converted.
y_pred.shape

In [ ]:
X_train

In [ ]:
X_train_X_y = pd.concat([X_train,y_train], axis=1)
X_train_X_y

In [ ]:
X_train_X_y['y_pred'] = y_pred
X_train_X_y

In [ ]:
# Error analysis
X_train_X_y.loc[(X_train_X_y['category']=='PARENTING') & (X_train_X_y['y_pred']=='IMPACT')]

In [ ]:
# Use the Naive Bayes to build a model
# Naive Bayes is a probabilistic classifier that works well with text data. 
# It makes use of Bayes' theorem to calculate the probability of a text 
# belonging to a particular category based on the presence of certain features 
# (words) in the text. Naive Bayes is known for its simplicity, efficiency, 
# and effectiveness in text classification tasks.
nb = NaiveBayes()
nb.fit(X_train_matrix.toarray(), y_train)

In [ ]:
# Change X and y to the dataset I want to evaluate the results in:
X_cm = X_train_matrix.toarray()
y_true_labels = y_train
model_nb = nb
# Apply trained model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Save the model
joblib.dump(nb, 'NaiveBayes_Model.joblib')

In [ ]:
y_train.value_counts()

In [ ]:
# y_pred is a numpy array. It needs to be converted.
y_pred.shape

In [ ]:
X_train_X_y_nb = pd.concat([X_train,y_train], axis=1)
X_train_X_y_nb

In [ ]:
X_train_X_y_nb['y_pred'] = y_pred
X_train_X_y_nb

In [ ]:
# Error analysis
X_train_X_y_nb.loc[(X_train_X_y_nb['category']=='PARENTING') & (X_train_X_y_nb['y_pred']=='IMPACT')]

## (4) Build model using deep learning

In [ ]:
# Load the train.csv and valid.csv.
train_data = pd.read_csv('train.csv', index_col=0)
valid_data = pd.read_csv('valid.csv', index_col=0)
train_data
valid_data

In [ ]:
# Remove the punctuations in train_data.
train_data['short_description'] = train_data['short_description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
train_data

In [ ]:
# Expand contractions
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

# Apply expansion to the 'short_description' column
train_data["short_description"] = train_data["short_description"].apply(expand_contractions)
train_data["short_description"]

In [ ]:
# Remove the punctuations in valid_data.
valid_data['short_description'] = valid_data['short_description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
valid_data

In [ ]:
valid_data["short_description"] = valid_data["short_description"].apply(expand_contractions)
valid_data["short_description"]

In [ ]:
sentences_train = train_data['short_description'].values
print(sentences_train)
sentences_valid = valid_data['short_description'].values
print(sentences_valid)
y_categories_train = y_train.values
print(y_categories_train)
y_categories_valid = y_valid.values
print(y_categories_valid)

In [ ]:
# Tokenize the text and covert it into numerical sequences.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences_train)
train_sequences = tokenizer.texts_to_sequences(sentences_train)
valid_sequences = tokenizer.texts_to_sequences(sentences_valid)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1 

In [ ]:
print(sentences_train[0])
print(train_sequences[0])

In [ ]:
print(max([len(x) for x in train_sequences]))

In [ ]:
# Pad the sequences to ensure consistent length.
max_length = 150
X_train_data = pad_sequences(train_sequences, maxlen=max_length, padding='post')
X_valid_data = pad_sequences(valid_sequences, maxlen=max_length, padding='post')

print(X_train_data[0, :])

In [ ]:
print(X_train_data.shape)
X_train_data

In [ ]:
print(X_valid_data.shape)
X_valid_data

In [ ]:
# Y train labels + Y valid labels.
encoder = sklearn.preprocessing.LabelEncoder()
category_train_labels = encoder.fit_transform(y_categories_train)
y_train_labels = category_train_labels.reshape((3862, 1))
print(y_train_labels)
category_valid_labels = encoder.fit_transform(y_categories_valid)
y_valid_labels = category_valid_labels.reshape((1538, 1))
print(y_valid_labels)

In [ ]:
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model.
history = model.fit(X_train_data, y_train_labels, validation_data=(X_valid_data, y_valid_labels), epochs=10, batch_size=32, verbose=True)

In [ ]:
# History saves the training into a dictionary structure with the keys below.
history.history.keys()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
# plt.ylim((-0.1, 1.1))
plt.show()

In [ ]:
category_train_labels

In [ ]:
np.asfarray(X_train_data)

In [ ]:
# Change X and y to the dataset I want to evaluate the results in:
X_cm = X_train_data
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
y_pred

In [ ]:
# Save the model
model.save('model.h5')

# TASK 3

## (1) Choose a metric to evaluate the model

In [ ]:
# Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# I chose the accuracy as the primary metric to evaluate my model.
# Calculate accuracy
accuracy = accuracy_score(y_true_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_true_labels, y_pred)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_true_labels, y_pred)
print("Recall:", recall)

# Calculate F1_score
f1 = f1_score(y_true_labels, y_pred)
print("F1-score:", f1)

In [ ]:
category_valid_labels

In [ ]:
print("******** Validation Data ********")
# Apply valid model to new dataset
X_cm_valid = X_valid_data
y_true_labels_valid = category_valid_labels
# Apply trained model to new dataset
y_pred_valid = model.predict(np.asfarray(X_cm_valid)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels_valid, y_pred_valid))
cm = confusion_matrix(y_true_labels_valid, y_pred_valid)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Evaluation
# Calculate accuracy
accuracy = accuracy_score(y_true_labels_valid, y_pred_valid)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_true_labels_valid, y_pred_valid)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_true_labels_valid, y_pred_valid)
print("Recall:", recall)

# Calculate F1_score
f1 = f1_score(y_true_labels_valid, y_pred_valid)
print("F1-score:", f1)

In [ ]:
# Comment:
'''
The choice of a good benchmark for this task depends on the specific dataset,
the complexity of the problem, and the performance achieved by existing models or
baselines. In this task, I think accuracy provides a general measure of the model's
performance.
'''

## (2) Evaluate the performance of each model

In [ ]:
# The First Model - DecisionTree
tree

In [ ]:
X_train_matrix

In [ ]:
y_train

In [ ]:
# Evaluate the train dataset.
X_cm = X_train_matrix
y_true_labels = y_train
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
y_valid

In [ ]:
# Evaluate the valid dataset.
X_cm = X_valid_matrix
y_true_labels = y_valid
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Comment:
# The performance of the tree model on the train dataset is significantly better
# than the validation dataset, indicating potential overfitting. The train set shows
# an accuracy of 100% with precision, recall, and F1-score all are 1. However, on the
# validation set, the accuracy drops to 78%. This suggests that the model have learned
# the specific patterns and nuances of the training data but struggle to generalize
# to unseen data. Further investigation is needed to address the overfitting issue,
# such as adjusting the modelcomplexity, regularization techniques, or increasing the
# amount of training data. It's important to ensure the model can perform well on
# real-world data, and additional evaluation and fine-tuning are recommended to improve
# their generalization capabilities.

In [ ]:
# The Second Model - Naive Bayes
nb = NaiveBayes()

In [ ]:
X_train_matrix

In [ ]:
y_train

In [ ]:
# Evaluate the train dataset.
X_cm = X_train_matrix.toarray()
y_true_labels = y_train
model_nb = nb
# Apply trained model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
X_valid_matrix

In [ ]:
y_valid

In [ ]:
# Evaluate the valid dataset.
X_cm = X_valid_matrix.toarray()
y_true_labels = y_valid
model_nb = nb
# Apply trained model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Comment:
# The performance of the nb model on the train dataset is significantly better
# than the validation dataset, indicating potential overfitting. The train set shows
# an accuracy of 95% with precision, recall, and F1-score all are 1. However, on the
# validation set, the accuracy drops to 75%. This suggests that the model have learned
# the specific patterns and nuances of the training data but struggle to generalize
# to unseen data. Further investigation is needed to address the overfitting issue,
# such as adjusting the modelcomplexity, regularization techniques, or increasing the
# amount of training data. It's important to ensure the model can perform well on
# real-world data, and additional evaluation and fine-tuning are recommended to improve
# their generalization capabilities.

In [ ]:
# The Third Model - Model created by myself
model

In [ ]:
X_train_data

In [ ]:
category_train_labels

In [ ]:
# Evaluate the train dataset.
X_cm = X_train_data
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Evaluate the valid dataset.
# Apply valid model to new dataset
X_cm_valid = X_valid_data
y_true_labels_valid = category_valid_labels
# Apply trained model to new dataset
y_pred_valid = model.predict(np.asfarray(X_cm_valid)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels_valid, y_pred_valid))
cm = confusion_matrix(y_true_labels_valid, y_pred_valid)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Comment:
# The performance of my model on the train dataset is significantly better
# than the validation dataset, indicating potential overfitting. The train set shows
# an accuracy of 100% with precision, recall, and F1-score all are 1. However, on the
# validation set, the accuracy drops to 84%. This suggests that the model have learned
# the specific patterns and nuances of the training data but struggle to generalize
# to unseen data. Further investigation is needed to address the overfitting issue,
# such as adjusting the modelcomplexity, regularization techniques, or increasing the
# amount of training data. It's important to ensure the model can perform well on
# real-world data, and additional evaluation and fine-tuning are recommended to improve
# their generalization capabilities.

## (3) Perform an error analysis

### Decision Tree Model

In [ ]:
# The First model
tree

In [ ]:
y_train

In [ ]:
y_train.value_counts()

In [ ]:
# model show
X_cm = X_train_matrix
y_true_labels = y_train
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Predicted y_train labels
y_pred = model_tree.predict(X_cm)

In [ ]:
y_pred.shape

In [ ]:
X_train_X_y = pd.concat([X_train,y_train], axis=1)
X_train_X_y

In [ ]:
X_train_X_y['y_pred'] = y_pred
X_train_X_y

In [ ]:
# Error analysis - train dataset
print(X_train_X_y.loc[(X_train_X_y['category']=='PARENTING') & (X_train_X_y['y_pred']=='IMPACT')])
print(X_train_X_y.loc[(X_train_X_y['category']=='IMPACT') & (X_train_X_y['y_pred']=='PARENTING')])

In [ ]:
# Model show - valid dataset
X_cm = X_valid_matrix
y_true_labels = y_valid
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
X_valid
y_valid

In [ ]:
X_valid_X_y = pd.concat([X_valid,y_valid], axis=1)
X_valid_X_y

In [ ]:
X_valid_X_y['y_pred'] = y_pred
X_valid_X_y

In [ ]:
# Error analysis - valid dataset
print(X_valid_X_y.loc[(X_valid_X_y['category']=='PARENTING') & (X_valid_X_y['y_pred']=='IMPACT')])
print(X_valid_X_y.loc[(X_valid_X_y['category']=='IMPACT') & (X_valid_X_y['y_pred']=='PARENTING')])

In [ ]:
# Comment:
'''
There are 1 error in train dataset and 202+128 errors in valid dataset
tested by decisiontree model.
'''

### Naive Bayes Model

In [ ]:
# The Second Model
nb

In [ ]:
# Evaluate the train dataset.
X_cm = X_train_matrix.toarray()
y_true_labels = y_train
model_nb = nb
# Apply trained model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
y_train

In [ ]:
X_train_X_y = pd.concat([X_train,y_train], axis=1)
X_train_X_y

In [ ]:
X_train_X_y['y_pred'] = y_pred
X_train_X_y

In [ ]:
# Error analysis - train dataset
print(X_train_X_y.loc[(X_train_X_y['category']=='PARENTING') & (X_train_X_y['y_pred']=='IMPACT')])
print(X_train_X_y.loc[(X_train_X_y['category']=='IMPACT') & (X_train_X_y['y_pred']=='PARENTING')])

In [ ]:
# Evaluate the valid dataset.
X_cm = X_valid_matrix.toarray()
y_true_labels = y_valid
model_nb = nb
# Apply valid model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
X_valid_X_y = pd.concat([X_valid,y_valid], axis=1)
X_valid_X_y
X_valid_X_y['y_pred'] = y_pred
X_valid_X_y

In [ ]:
# Error analysis - valid dataset
X_valid_X_y.loc[(X_valid_X_y['category']=='PARENTING') & (X_valid_X_y['y_pred']=='IMPACT')]
X_valid_X_y.loc[(X_valid_X_y['category']=='IMPACT') & (X_valid_X_y['y_pred']=='PARENTING')]

In [ ]:
# Comment:
'''
There are 185 errors in train dataset and 146+233 errors in valid dataset
tested by naive bayes model.
'''

### My Own Model

In [ ]:
# The Third Model
model

In [ ]:
# Evaluate the train dataset.
X_cm = X_train_data
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
y_pred_list = y_pred.tolist()
y_pred_list = sum(y_pred_list,[])
y_pred_renewed = []
for i in y_pred_list:
    if i == 0:
        y_pred_renewed.append('IMPACT')
    elif i == 1:
        y_pred_renewed.append('PARENTING')

In [ ]:
X_train_X_y = pd.concat([X_train,y_train], axis=1)
X_train_X_y
X_train_X_y['y_pred'] = y_pred_renewed
X_train_X_y

In [ ]:
# Error analysis - train dataset
X_train_X_y.loc[(X_train_X_y['category']=='PARENTING') & (X_train_X_y['y_pred']=='IMPACT')]
X_train_X_y.loc[(X_train_X_y['category']=='IMPACT') & (X_train_X_y['y_pred']=='PARENTING')]

In [ ]:
# Evaluate the valid dataset.
# Apply valid model to new dataset
X_cm_valid = X_valid_data
y_true_labels_valid = category_valid_labels
# Apply trained model to new dataset
y_pred_valid = model.predict(np.asfarray(X_cm_valid)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels_valid, y_pred_valid))
cm = confusion_matrix(y_true_labels_valid, y_pred_valid)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Renew the element in y_pred into 'IMPACT' or 'PARENTING'
y_pred_list = y_pred_valid.tolist()
y_pred_list = sum(y_pred_list,[])
y_pred_renewed = []
for i in y_pred_list:
    if i == 0:
        y_pred_renewed.append('IMPACT')
    elif i == 1:
        y_pred_renewed.append('PARENTING')

In [ ]:
X_valid_X_y = pd.concat([X_valid,y_valid], axis=1)
X_valid_X_y
X_valid_X_y['y_pred'] = y_pred_renewed
X_valid_X_y

In [ ]:
# Error analysis - valid dataset
print(X_valid_X_y.loc[(X_valid_X_y['category']=='PARENTING') & (X_valid_X_y['y_pred']=='IMPACT')])
print(X_valid_X_y.loc[(X_valid_X_y['category']=='IMPACT') & (X_valid_X_y['y_pred']=='PARENTING')])

In [ ]:
# Comment:
'''
There is 0 error in train dataset and 265+12 errors in valid dataset
tested by my own model.
'''

In [ ]:
# From the analysis, I have learned that the models, while achieving reasonably
# good performance on the training set, struggle when applied to the validation
# set. This indicates a certain degree of overfitting, where the models have become
# too specialized in capturing the specific patterns present in the training data
# and are unable to generalize well to unseen examples.

## (4) Change each model and redo

In [ ]:
# Define the showing graph function.
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
# Remove stop_words
def remove_stopwords(text):
    en_stops = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    removed_text = ' '.join([w for w in word_tokens if not w in en_stops])
    return removed_text

In [ ]:
# Lowercase
def lowercase(text):
    word_tokens = word_tokenize(text)
    lower_words = ' '.join([word.lower() for word in word_tokens])
    return lower_words

In [ ]:
# Stemming
def stemming(text):
    porter_stemmer=PorterStemmer()
    word_tokens = word_tokenize(text)
    stemmed_words = ' '.join([porter_stemmer.stem(word=word) for word in word_tokens])
    return stemmed_words

In [ ]:
# Lemmatization
def lemmatize_all(text):
    wnl = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    tagged_sent = nltk.pos_tag(word_tokens)
    for word, tag in tagged_sent:
        if tag.startswith('NN'):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
        else:
            yield word
def merge(text):
    all_words = lemmatize_all(text)
    return ' '.join(all_words)

In [ ]:
# Expand function
def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

### I: DecisionTree Model

#### Tree Previous Model

In [ ]:
# Previous Model
tree = DecisionTreeClassifier()
tree.fit(X_train_matrix, y_train)

In [ ]:
# Training Dataset
X_cm = X_train_matrix.toarray()
y_true_labels = y_train
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset
X_cm = X_valid_matrix
y_true_labels = y_valid
model_tree = tree
# Apply trained model to new dataset
y_pred = model_tree.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

#### Tree New Model

In [ ]:
# Load the train.csv and valid.csv.
train_data = pd.read_csv('train.csv', index_col=0)
valid_data = pd.read_csv('valid.csv', index_col=0)
X_train = train_data['short_description']
X_train

In [ ]:
# Text Processing
train_data['short_description'] = train_data['short_description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
train_data['short_description'] = train_data['short_description'].apply(lowercase)
train_data['short_description'] = train_data['short_description'].apply(expand_contractions)
train_data['short_description'] = train_data['short_description'].apply(merge)
X_train_processed = train_data['short_description']
X_train_processed

In [ ]:
# Preprocess the text data.
vectorizer_tree_improve = CountVectorizer(stop_words = 'english')
vectorizer_tree_improve.fit(X_train_processed)

In [ ]:
X_train_matrix_processed = vectorizer_tree_improve.transform(X_train_processed)
X_train_matrix_processed

In [ ]:
tree_improve_model = DecisionTreeClassifier()
tree_improve_model.fit(X_train_matrix_processed, y_train)

In [ ]:
# Training Dataset
X_cm = X_train_matrix_processed
y_true_labels = y_train
# Apply trained model to new dataset
y_pred = tree_improve_model.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
X_valid = valid_data['short_description']
X_valid

In [ ]:
# Text Processing
valid_data['short_description'] = valid_data['short_description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
valid_data['short_description'] = valid_data['short_description'].apply(lowercase)
valid_data['short_description'] = valid_data['short_description'].apply(expand_contractions)
valid_data['short_description'] = valid_data['short_description'].apply(merge)
X_valid_processed = valid_data['short_description']
X_valid_processed

In [ ]:
X_valid_matrix_processed = vectorizer_tree_improve.transform(X_valid_processed)
X_valid_matrix_processed

In [ ]:
# Validing Dataset
X_cm = X_valid_matrix_processed
y_true_labels = y_valid
y_pred = tree_improve_model.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

#### Save Model

In [ ]:
tree_improve_model

In [ ]:
# Save model
joblib.dump(tree_improve_model, 'DecisionTree_Improve_Model.joblib')

### II: Naive Bayes Model

#### Bayes Previous Model

In [ ]:
# Previous Model
nb

In [ ]:
# Training Dataset
X_cm = X_train_matrix.toarray()
y_true_labels = y_train
model_nb = nb
# Apply trained model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset
X_cm = X_valid_matrix.toarray()
y_true_labels = y_valid
model_nb = nb
# Apply trained model to new dataset
y_pred = model_nb.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

#### New Model

In [ ]:
# Processed Text
X_train_matrix_processed
X_valid_matrix_processed
y_train
y_valid

In [ ]:
# Build new model
nb_improve_model = NaiveBayes()
nb_improve_model.fit(X_train_matrix_processed.toarray(), y_train)

In [ ]:
# Training Dataset
X_cm = X_train_matrix_processed.toarray()
y_true_labels = y_train
y_pred = nb_improve_model.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset
X_cm = X_valid_matrix_processed.toarray()
y_true_labels = y_valid
y_pred = nb_improve_model.predict(X_cm)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot = True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

#### Save Model

In [ ]:
nb_improve_model

In [ ]:
# Save model
joblib.dump(nb_improve_model, 'NaiveBayes_Improve_Model.joblib')

### III: My Own Model

#### Previous Model

In [ ]:
# Previous Model
model

In [ ]:
history.history.keys()

In [ ]:
# Training Dataset.
X_cm = X_train_data
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset.
X_cm = X_valid_data
y_true_labels = category_valid_labels
# Apply trained model to new dataset
y_pred = model.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Previous Accuracy
# history = model.fit(X_train_data, y_train_labels, validation_data=(X_valid_data, y_valid_labels), epochs=10, batch_size=32, verbose=False)
loss, accuracy = model.evaluate(X_train_data, y_train_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_valid_data, y_valid_labels, verbose=False)
print("Validing Accuracy: {:.4f}".format(accuracy))
plot_history(history)

#### New Model 1

In [ ]:
# Improving Text
X_train_processed
X_valid_processed
y_train_labels
y_valid_labels
category_train_labels
category_valid_labels

In [ ]:
sentences_train_processed = X_train_processed.values
print(sentences_train_processed)
sentences_valid_processed = X_valid_processed.values
print(sentences_valid_processed)

In [ ]:
# Tokenize the text and covert it into numerical sequences.
tokenizer_processed = Tokenizer()
tokenizer_processed.fit_on_texts(sentences_train_processed)
train_sequences_processed = tokenizer_processed.texts_to_sequences(sentences_train_processed)
valid_sequences_processed = tokenizer_processed.texts_to_sequences(sentences_valid_processed)

# Adding 1 because of reserved 0 index
vocab_size_train = len(tokenizer_processed.word_index) + 1 

In [ ]:
len(tokenizer_processed.word_index) + 1

In [ ]:
print(max([len(x) for x in train_sequences_processed]))

In [ ]:
# Pad the sequences to ensure consistent length.
max_length = 150
X_train_data_processed = pad_sequences(train_sequences_processed, maxlen=max_length, padding='post')
X_valid_data_processed = pad_sequences(valid_sequences_processed, maxlen=max_length, padding='post')

print(X_train_data_processed[0, :])

In [ ]:
embedding_dim = 50

model_improve1 = Sequential()
model_improve1.add(Embedding(input_dim=vocab_size_train, output_dim=embedding_dim, input_length=max_length))
model_improve1.add(Flatten())
model_improve1.add(Dense(10, activation='relu'))
model_improve1.add(Dense(1, activation='sigmoid'))
model_improve1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_improve1.summary()

In [ ]:
# Train the model.
history_improved1 = model_improve1.fit(X_train_data_processed, y_train_labels, validation_data=(X_valid_data_processed, y_valid_labels), epochs=10, batch_size=32, verbose=True)

In [ ]:
# History saves the training into a dictionary structure with the keys below.
history_improved1.history.keys()

In [ ]:
# summarize history for accuracy
plt.plot(history_improved1.history['accuracy'])
plt.plot(history_improved1.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

In [ ]:
# Training Dataset
X_cm = X_train_data_processed
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model_improve1.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset
X_cm = X_valid_data_processed
y_true_labels = category_valid_labels
# Apply trained model to new dataset
y_pred = model_improve1.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

#### New Model 2

In [ ]:
# Buidling a Better Model
embedding_dim = 100

model_improve2 = Sequential()
model_improve2.add(Embedding(input_dim=vocab_size_train, output_dim=embedding_dim, input_length=max_length))
model_improve2.add(GlobalAveragePooling1D())
model_improve2.add(Dense(10, activation='relu'))
model_improve2.add(Dense(1, activation='sigmoid'))
model_improve2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_improve2.summary()

In [ ]:
# History
history_improve2 = model_improve2.fit(X_train_data_processed, y_train_labels, validation_data=(X_valid_data_processed, y_valid_labels), epochs=10, batch_size=32, verbose=False)

In [ ]:
# Improving Model
loss, accuracy = model_improve2.evaluate(X_train_data_processed, y_train_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_improve2.evaluate(X_valid_data_processed, y_valid_labels, verbose=False)
print("Validing Accuracy: {:.4f}".format(accuracy))
plot_history(history_improve2)

In [ ]:
# Training Dataset
X_cm = X_train_data_processed
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model_improve2.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset.
X_cm = X_valid_data_processed
y_true_labels = category_valid_labels
# Apply trained model to new dataset
y_pred = model_improve2.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Save Model
model_improve2.save('model_improve2.h5')

In [ ]:
# Load Model
best_model = load_model('model_improve2.h5')

#### New Model 3

In [ ]:
# Imrpoving model again
embedding_dim = 100

model_improve3 = Sequential()
model_improve3.add(Embedding(input_dim=vocab_size_train, output_dim=embedding_dim, input_length=max_length))
model_improve3.add(Conv1D(128, 5, activation='relu'))
model_improve3.add(GlobalAveragePooling1D())
model_improve3.add(Dense(10, activation='relu'))
model_improve3.add(Dense(1, activation='sigmoid'))
model_improve3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_improve3.summary()

In [ ]:
# History
history_improve3 = model_improve3.fit(X_train_data_processed, y_train_labels, validation_data=(X_valid_data_processed, y_valid_labels), epochs=10, batch_size=32, verbose=False)

In [ ]:
# Improving Model again
loss, accuracy = model_improve3.evaluate(X_train_data_processed, y_train_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_improve3.evaluate(X_valid_data_processed, y_valid_labels, verbose=False)
print("Validing Accuracy: {:.4f}".format(accuracy))
plot_history(history_improve3)

In [ ]:
# Training Dataset
X_cm = X_train_data_processed
y_true_labels = category_train_labels
# Apply trained model to new dataset
y_pred = model_improve3.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Validing Dataset.
X_cm = X_valid_data_processed
y_true_labels = category_valid_labels
# Apply trained model to new dataset
y_pred = model_improve3.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

## (5) Cross Validation

In [ ]:
# Load the train.csv and valid.csv.
train_data = pd.read_csv('train.csv', index_col=0)
valid_data = pd.read_csv('valid.csv', index_col=0)
train_data
#valid_data

In [ ]:
# Merge train and validation sets
X_merged = np.concatenate((train_data, valid_data))
y_merged = np.concatenate((y_train, y_valid))

In [ ]:
X_merged_df = pd.DataFrame(X_merged, columns = ['short_description'])
X_merged_df
y_merged_df = pd.DataFrame(y_merged, columns = ['category'])
y_merged_df

In [ ]:
# Text Processing
X_merged_df["short_description"] = X_merged_df["short_description"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
X_merged_df["short_description"] = X_merged_df["short_description"].apply(lowercase)
X_merged_df["short_description"] = X_merged_df["short_description"].apply(expand_contractions)
X_merged_df["short_description"] = X_merged_df["short_description"].apply(merge)
X_merged_processed = X_merged_df["short_description"]
X_merged_processed

In [ ]:
sentences_merged = X_merged_processed.values
print(sentences_merged)
categories_merged = y_merged_df['category'].values
print(categories_merged)

In [ ]:
# Tokenize the text and covert it into numerical sequences.
tokenizer_merged = Tokenizer()
tokenizer_merged.fit_on_texts(sentences_merged)
merged_sequences = tokenizer_merged.texts_to_sequences(sentences_merged)

In [ ]:
vocab_size_merged = len(tokenizer_merged.word_index) + 1 
vocab_size_merged

In [ ]:
print(sentences_merged[0])
print(merged_sequences[0])

In [ ]:
# Pad the sequences to ensure consistent length.
max_length = 150
merged_data = pad_sequences(merged_sequences, maxlen=max_length, padding='post')
print(merged_data)
#print(merged_data[0, :])

In [ ]:
# y_merged labels
encoder_merged = sklearn.preprocessing.LabelEncoder()
category_merged_labels = encoder_merged.fit_transform(categories_merged)
merged_labels = category_merged_labels.reshape((5400, 1))
print(category_merged_labels)

### DecisionTree Improved Model

In [ ]:
# Load model
model_decisiontree_improve = joblib.load('DecisionTree_Improve_Model.joblib')

In [ ]:
# Perform cross-validation
scores = cross_val_score(model_decisiontree_improve, merged_data, merged_labels, cv=5, error_score='raise')

In [ ]:
scores

### Naive Bayes Improved Model

In [ ]:
# Load model
model_naivebayes_improve = joblib.load('NaiveBayes_Improve_Model.joblib')

In [ ]:
# Perform cross-validation
scores = cross_val_score(model_naivebayes_improve, merged_data, merged_labels, cv=5, error_score='raise')

In [ ]:
scores

### My Improved Model

In [ ]:
# Load Model
best_model = load_model('model_improve2.h5')

In [ ]:
def create_model():
    embedding_dim = 100
    model_my_final = Sequential()
    model_my_final.add(Embedding(input_dim=vocab_size_merged, output_dim=embedding_dim, input_length=max_length))
    model_my_final.add(Conv1D(128, 5, activation='relu'))
    model_my_final.add(GlobalAveragePooling1D())
    model_my_final.add(Dense(10, activation='relu'))
    model_my_final.add(Dense(1, activation='sigmoid'))
    model_my_final.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_my_final

In [ ]:
model_f = KerasClassifier(model=create_model, epochs=10, batch_size=32)

In [ ]:
# Perform cross-validation
scores = cross_val_score(model_f, merged_data, merged_labels, cv=5, error_score='raise')

In [ ]:
scores

In [ ]:
# Comment
'''
Based on the provided socres, it appears that the model is performing fairly well.
The average accuracy across the five folds of cross-validation is around 83%-84%,
which indicates that the model is able to classify the articles with a good level
of accuracy.

However, it is important to note that accuracy alone may not provide a complete picture
of the model's performance, especially if the dataset is imbalanced or if there are
specific classes that are more important to classify correctly than others.Therefore, 
it's recommended to consider other evaluation metrics such as precision, recall, 
and F1-score to assess the model's performance more comprehensively.
'''

## (6) Apply to Test.csv

In [ ]:
# Load the test.csv.
test_data = pd.read_csv('test.csv', index_col=0)
test_data

In [ ]:
# Check if there are blank values in the dataset.
test_data['short_description'].isnull().sum()

In [ ]:
y_test

In [ ]:
# Text Processing
test_data['short_description'] = test_data['short_description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
test_data['short_description'] = test_data['short_description'].apply(lowercase)
test_data['short_description'] = test_data['short_description'].apply(expand_contractions)
test_data['short_description'] = test_data['short_description'].apply(merge)
X_test_processed = test_data['short_description']
X_test_processed

In [ ]:
X_test_matrix_processed = vectorizer_tree_improve.transform(X_test_processed)
X_test_matrix_processed

#### My Own Model

In [ ]:
# Load Model
best_model = load_model('model_improve2.h5')

In [ ]:
sentences_test_processed = X_test_processed.values

In [ ]:
test_sequences_processed = tokenizer_processed.texts_to_sequences(sentences_test_processed)
# Pad the sequences to ensure consistent length.
max_length = 150
X_test_data_processed = pad_sequences(test_sequences_processed, maxlen=max_length, padding='post')
X_test_data_processed

In [ ]:
y_categories_test = y_test.values

In [ ]:
# Y test labels.
encoder = sklearn.preprocessing.LabelEncoder()
category_test_labels = encoder.fit_transform(y_categories_test)
y_test_labels = category_test_labels.reshape((2318, 1))
print(y_test_labels)

In [ ]:
# Testing Dataset.
X_cm = X_test_data_processed
y_true_labels = category_test_labels
# Apply trained model to new dataset
y_pred = best_model.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Comment:
'''
The results of accuracy is similar to the one I obtained for the validation set.
'''

## (7) Retrain the best model

In [ ]:
# Train and Valid data merge
X_merged_df = pd.DataFrame(X_merged, columns = ['short_description'])
X_merged_df

In [ ]:
y_merged_df = pd.DataFrame(y_merged, columns = ['category'])
y_merged_df

In [ ]:
sentences_valid_processed

In [ ]:
y_valid_labels

In [ ]:
# Text Preprocessing
X_merged_df["short_description"] = X_merged_df["short_description"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
X_merged_df["short_description"] = X_merged_df["short_description"].apply(lowercase)
X_merged_df["short_description"] = X_merged_df["short_description"].apply(expand_contractions)
X_merged_df["short_description"] = X_merged_df["short_description"].apply(merge)
X_merged_processed = X_merged_df["short_description"]
X_merged_processed

In [ ]:
sentences_merged = X_merged_processed.values
print(sentences_merged)
categories_merged = y_merged_df['category'].values
print(categories_merged)

In [ ]:
# Tokenize the text and covert it into numerical sequences.
tokenizer_merged = Tokenizer()
tokenizer_merged.fit_on_texts(sentences_merged)
merged_sequences = tokenizer_merged.texts_to_sequences(sentences_merged)
valid_sequences = tokenizer_merged.texts_to_sequences(sentences_valid_processed)

In [ ]:
vocab_size_merged = len(tokenizer_merged.word_index) + 1 
vocab_size_merged

In [ ]:
print(max([len(x) for x in merged_sequences]))

In [ ]:
# Pad the sequences to ensure consistent length.
max_length = 150
merged_data = pad_sequences(merged_sequences, maxlen=max_length, padding='post')
valid_data = pad_sequences(valid_sequences, maxlen=max_length, padding='post')
print(merged_data)
#print(merged_data[0, :])

In [ ]:
# y_merged labels
encoder_merged = sklearn.preprocessing.LabelEncoder()
category_merged_labels = encoder_merged.fit_transform(categories_merged)
merged_labels = category_merged_labels.reshape((5400, 1))
print(category_merged_labels)
category_valid_labels = encoder_merged.fit_transform(y_categories_valid)
valid_labels = category_valid_labels.reshape((1538, 1))
print(category_valid_labels)

In [ ]:
# Buidling a Better Model
embedding_dim = 100

model_retrain = Sequential()
model_retrain.add(Embedding(input_dim=vocab_size_merged, output_dim=embedding_dim, input_length=max_length))
model_retrain.add(GlobalAveragePooling1D())
model_retrain.add(Dense(10, activation='relu'))
model_retrain.add(Dense(1, activation='sigmoid'))
model_retrain.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_retrain.summary()

In [ ]:
# History
history_retrain = model_retrain.fit(merged_data, merged_labels, validation_data=(valid_data, valid_labels), epochs=10, batch_size=32, verbose=False)

In [ ]:
# Test data
# Load the test.csv.
test_data = pd.read_csv('test.csv', index_col=0)
test_data

In [ ]:
y_test

In [ ]:
# Test data preprocessing
test_data['short_description'] = test_data['short_description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
test_data['short_description'] = test_data['short_description'].apply(lowercase)
test_data['short_description'] = test_data['short_description'].apply(expand_contractions)
test_data['short_description'] = test_data['short_description'].apply(merge)
X_test_processed = test_data['short_description']
X_test_processed

In [ ]:
sentences_test_processed = X_test_processed.values

In [ ]:
# Test data preprocessing
test_sequences_processed = tokenizer_merged.texts_to_sequences(sentences_test_processed)
# Pad the sequences to ensure consistent length.
max_length = 150
X_test_data_processed = pad_sequences(test_sequences_processed, maxlen=max_length, padding='post')
X_test_data_processed

In [ ]:
y_categories_test = y_test.values

In [ ]:
# Y test labels.
encoder = sklearn.preprocessing.LabelEncoder()
category_test_labels = encoder.fit_transform(y_categories_test)
y_test_labels = category_test_labels.reshape((2318, 1))
print(y_test_labels)

In [ ]:
# Testing Dataset.
X_cm = X_test_data_processed
y_true_labels = category_test_labels
# Apply trained model to new dataset
y_pred = model_retrain.predict(np.asfarray(X_cm)).round()
#y_pred = (y_pred > 0.5).astype(int)
#classes = np.argmax(y_pred, axis=1)
#metrics.f1_score(y_true_labels, y_pred,average='weighted',zero_division=0)
print(metrics.classification_report(y_true_labels, y_pred))
cm = confusion_matrix(y_true_labels, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

In [ ]:
# Comment:
'''
The accuracy of one category of the retrained model seems to be improved 
from 62% to 69%.
'''

## (8) Files

In [ ]:
# Requirements.txt